# 要約 
このJupyter Notebookでは、Kaggleの「LMSYS - Chatbot Arena」コンペティションにおいて、人間の好みに基づいたモデルのトレーニングおよび評価を行うための準備と実行が取り扱われています。具体的には、異なる応答を提示するチャットボットのモデルを比較し、どちらが好まれるかを予測するための機械学習モデルの構築を目指しています。

### 取り組んでいる問題
Notebookは、ユーザープロンプトに対する2つの異なる応答の好みを判定する「報酬モデル」を構築するという課題に取り組んでいます。具体的には、トレーニングデータを処理し、必要な特徴量を抽出し、最終的にモデルのトレーニングを行う過程を示しています。

### 主な手法とライブラリ
1. **データ読み込みと前処理**:
   - Pandasを使用してトレーニングデータおよびテストデータをCSVファイルから読み込み、各応答に関する特徴を処理するカスタム関数が定義されています。
   - Hugging Faceの`datasets`ライブラリを利用してデータセットを管理します。

2. **モデルの選定と設定**:
   - Hugging FaceのTransformersライブラリから、Mistralという事前学習済みの因果言語モデルを使用し、量子化設定を行った上でモデルを読み込みます。
   - LoRA（Low-Rank Adaptation）技術を用いて、モデルにファインチューニングする設定が適用されます。

3. **トレーニング**:
   - `Trainer`クラスを拡張してカスタムトレーナーを定義し、損失計算にクラス重みを用いるなどの工夫がされています。
   - モデルのトレーニングは、PyTorchを使用し、勾配チェックポイント機能や、評価メトリクスを定義して行います。

4. **評価**:
   - トレーニング後、ユーザーが提示したプロンプトに基づいてモデルがどのように応答するかを生成し、その性能を評価します。

このNotebookは、データの前処理からモデルのトレーニングと評価に至る一連の流れを詳細に記述し、実践的な機械学習プロジェクトの進め方を示しています。使用するライブラリには、NumPy、Pandas、Transformers、Hugging FaceのDatasets、PyTorch、PEFT（Parameter-Efficient Fine-Tuning）などがあります。

---


# 用語概説 
以下は、そのJupyter Notebook内の機械学習および深層学習関連の専門用語の簡単な解説です。特に、初心者がつまずきやすいものに焦点を当てています。

1. **量子化 (Quantization)**:
   - モデルのサイズを削減し、推論速度を向上させるために、モデルの重みを低精度（例：32ビットから8ビットや4ビット）に変換するプロセスです。これにより、メモリ使用量や計算コストが減少します。

2. **ダブル量子化 (Double Quantization)**:
   - 通常の量子化に加え、さらに別の段階で量子化を行うことです。この手法は、さらなる圧縮を実現するために使用されます。

3. **bfloat16 (Brain Floating Point)**:
   - Googleが開発した浮動小数点数形式で、深層学習の計算でよく使用されます。従来の16ビット浮動小数点よりも大きな範囲を持ちながら、表現精度をある程度保持します。

4. **Gradient Checkpointing (勾配チェックポイント)**:
   - メモリを節約する手法で、順伝播中に計算された中間結果を保持せず、逆伝播中に必要な部分を再計算することでメモリ使用量を削減します。

5. **クラス重み (Class Weights)**:
   - 不均衡なデータセットにおいて、特定のクラスの影響を強調または抑えるために、損失関数に独自の重みを設定することです。これにより、少数クラスの学習を改善します。

6. **トレーニング可能なパラメータ (Trainable Parameters)**:
   - モデルのトレーニングによって更新されるパラメータのことを指します。これには、モデルの重みやバイアスが含まれます。

7. **PEFT (Parameter-Efficient Fine-Tuning)**:
   - モデルの全体を微調整するのではなく、特定の部分のみを微調整する手法で、少ないパラメータで効果的なトレーニングを実現します。

8. **LoRA (Low-Rank Adaptation)**:
   - PEFTの一種で、モデルの重みを低ランクで近似する手法です。これにより計算コストを削減し、効率的なファインチューニングが可能となります。

9. **FSDP (Fully Sharded Data Parallel)**:
   - モデルのパラメータを完全に分割し、複数のデバイスで並行してトレーニングしたりチェックポイントを保存したりするための手法です。これにより、メモリ効率が向上します。

10. **Data Collator (データコレータ)**:
    - トレーニングデータをバッチ処理する際に、必要なフォーマットに整形するための関数やオブジェクトです。特に、パディングやマスキングを考慮する必要があります。

11. **ウォームアップステップ (Warmup Steps)**:
    - トレーニングの初期段階で、学習率を徐々に増加させるためのステップです。これにより、モデルが安定し、トレーニングの初期の不安定さを軽減します。

これらの用語は、特に初心者が実務体験がない場合に馴染みが薄い範囲で、深層学習モデルやトレーニングプロセスを理解する上で重要です。

---


<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
```

</div>
<div class="column-right">

# 日本語訳

```python
# このPython 3環境には、多くの便利な分析ライブラリがインストールされています
# これはkaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# たとえば、いくつかの便利なパッケージを読み込むことができます

import numpy as np # 線形代数用
import pandas as pd # データ処理、CSVファイルの入出力（例: pd.read_csv）

# 入力データファイルは、読み取り専用の"../input/"ディレクトリにあります
# たとえば、これを実行することで（実行をクリックするか、Shift + Enterを押すことで）入力ディレクトリ内のすべてのファイルをリスト表示できます

import os # オペレーティングシステムに関する機能を使用するためのモジュール
for dirname, _, filenames in os.walk('/kaggle/input'): # "/kaggle/input"ディレクトリ内のファイルやフォルダを再帰的に巡回します
    for filename in filenames: # ファイル名のリストをループします
        print(os.path.join(dirname, filename)) # 各ファイルのフルパスを出力します

# あなたは現在のディレクトリ（/kaggle/working/）に最大20GBを保存できます。これは「すべて保存して実行」機能を使用してバージョンを作成したときに出力として保持されます
# また、一時ファイルを/kaggle/temp/に書き込むこともできますが、現在のセッションの外では保存されません
```

</div>
</details>

In [ ]:
# このPython 3環境には、多くの便利な分析ライブラリがインストールされています
# これはkaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# たとえば、いくつかの便利なパッケージを読み込むことができます

import numpy as np # 線形代数用
import pandas as pd # データ処理、CSVファイルの入出力（例: pd.read_csv）

# 入力データファイルは、読み取り専用の"../input/"ディレクトリにあります
# たとえば、これを実行することで（実行をクリックするか、Shift + Enterを押すことで）入力ディレクトリ内のすべてのファイルをリスト表示できます

import os # オペレーティングシステムに関する機能を使用するためのモジュール
for dirname, _, filenames in os.walk('/kaggle/input'): # "/kaggle/input"ディレクトリ内のファイルやフォルダを再帰的に巡回します
    for filename in filenames: # ファイル名のリストをループします
        print(os.path.join(dirname, filename)) # 各ファイルのフルパスを出力します

# あなたは現在のディレクトリ（/kaggle/working/）に最大20GBを保存できます。これは「すべて保存して実行」機能を使用してバージョンを作成したときに出力として保持されます
# また、一時ファイルを/kaggle/temp/に書き込むこともできますが、現在のセッションの外では保存されません

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# !pip install cupy-cuda11x>=12.0.0
# !pip install packaging>=22
# !pip install shapely>=2.0.1
# !pip install numpy<1.26
# !pip install scipy<1.12
!pip install transformers datasets accelerate peft

```

</div>
<div class="column-right">

# 日本語訳

```python
# !pip install cupy-cuda11x>=12.0.0  # GPUを利用したCuPyライブラリのインストール（CUDA 11.x以上）
# !pip install packaging>=22  # パッケージ管理用ライブラリのインストール（バージョン22以上）
# !pip install shapely>=2.0.1  # 幾何学的操作を行うためのShapelyライブラリのインストール（バージョン2.0.1以上）
# !pip install numpy<1.26  # 数値計算用のNumPyライブラリのインストール（バージョン1.26未満）
# !pip install scipy<1.12  # 科学計算用のSciPyライブラリのインストール（バージョン1.12未満）
!pip install transformers datasets accelerate peft  # Transformers、データセット、加速化、PEFTライブラリのインストール
```

</div>
</details>

In [ ]:
# !pip install cupy-cuda11x>=12.0.0  # GPUを利用したCuPyライブラリのインストール（CUDA 11.x以上）
# !pip install packaging>=22  # パッケージ管理用ライブラリのインストール（バージョン22以上）
# !pip install shapely>=2.0.1  # 幾何学的操作を行うためのShapelyライブラリのインストール（バージョン2.0.1以上）
# !pip install numpy<1.26  # 数値計算用のNumPyライブラリのインストール（バージョン1.26未満）
# !pip install scipy<1.12  # 科学計算用のSciPyライブラリのインストール（バージョン1.12未満）
!pip install transformers datasets accelerate peft  # Transformers、データセット、加速化、PEFTライブラリのインストール

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# You only need to run this once per machine
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q -U datasets scipy ipywidgets
!pip install datasets
```

</div>
<div class="column-right">

# 日本語訳

```python
# このコマンドは、マシンごとに一度だけ実行すれば十分です
!pip install -q -U bitsandbytes  # bitsandbytesライブラリのインストール（省略モードで、アップグレードする）
# !pip install -q -U git+https://github.com/huggingface/transformers.git  # Hugging FaceのTransformersライブラリをGitHubからインストール（アップグレード）
# !pip install -q -U git+https://github.com/huggingface/peft.git  # Hugging FaceのPEFTライブラリをGitHubからインストール（アップグレード）
# !pip install -q -U git+https://github.com/huggingface/accelerate.git  # Hugging FaceのAccelerateライブラリをGitHubからインストール（アップグレード）
# !pip install -q -U datasets scipy ipywidgets  # データセット、SciPy、およびipywidgetsライブラリのインストール（アップグレード）
!pip install datasets  # datasetsライブラリのインストール
```

</div>
</details>

In [ ]:
# このコマンドは、マシンごとに一度だけ実行すれば十分です
!pip install -q -U bitsandbytes  # bitsandbytesライブラリのインストール（省略モードで、アップグレードする）
# !pip install -q -U git+https://github.com/huggingface/transformers.git  # Hugging FaceのTransformersライブラリをGitHubからインストール（アップグレード）
# !pip install -q -U git+https://github.com/huggingface/peft.git  # Hugging FaceのPEFTライブラリをGitHubからインストール（アップグレード）
# !pip install -q -U git+https://github.com/huggingface/accelerate.git  # Hugging FaceのAccelerateライブラリをGitHubからインストール（アップグレード）
# !pip install -q -U datasets scipy ipywidgets  # データセット、SciPy、およびipywidgetsライブラリのインストール（アップグレード）
!pip install datasets  # datasetsライブラリのインストール

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from datasets import load_dataset

train_dataset = load_dataset('csv', data_files='/kaggle/input/lmsys-chatbot-arena/test.csv')
```

</div>
<div class="column-right">

# 日本語訳

```python
from datasets import load_dataset  # datasetsモジュールからload_dataset関数をインポートします

# 'csv'形式のデータセットを読み込みます。特に'test.csv'ファイルを指定しています。
train_dataset = load_dataset('csv', data_files='/kaggle/input/lmsys-chatbot-arena/test.csv')  # テストデータを含むCSVファイルを読み込み、train_datasetに格納します
```

</div>
</details>

In [ ]:
from datasets import load_dataset  # datasetsモジュールからload_dataset関数をインポートします

# 'csv'形式のデータセットを読み込みます。特に'test.csv'ファイルを指定しています。
train_dataset = load_dataset('csv', data_files='/kaggle/input/lmsys-chatbot-arena/test.csv')  # テストデータを含むCSVファイルを読み込み、train_datasetに格納します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from huggingface_hub import notebook_login

notebook_login()
```

</div>
<div class="column-right">

# 日本語訳

```python
from huggingface_hub import notebook_login  # Hugging Face Hubからnotebook_login関数をインポートします

# Hugging Face Hubへのログインプロンプトを表示します
notebook_login()  # ユーザーがHugging Face Hubにログインできるようにします。これにより、モデルやデータセットにアクセスできるようになります。
```

</div>
</details>

In [ ]:
from huggingface_hub import notebook_login  # Hugging Face Hubからnotebook_login関数をインポートします

# Hugging Face Hubへのログインプロンプトを表示します
notebook_login()  # ユーザーがHugging Face Hubにログインできるようにします。これにより、モデルやデータセットにアクセスできるようになります。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config,device_map="auto")
```

</div>
<div class="column-right">

# 日本語訳

```python
import torch  # PyTorchライブラリをインポートします
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig  # Hugging FaceのTransformersライブラリから必要なクラスをインポートします

base_model_id = "mistralai/Mistral-7B-v0.1"  # 使用する基本モデルのIDを指定します
bnb_config = BitsAndBytesConfig(  # 量子化に関する設定を作成します
    load_in_4bit=True,  # 4ビットでモデルを読み込む設定をします
    bnb_4bit_use_double_quant=True,  # ダブル量子化を使用する設定を行います
    bnb_4bit_quant_type="nf4",  # 量子化のタイプを指定します（nf4を使用）
    bnb_4bit_compute_dtype=torch.bfloat16  # 計算データ型をbfloat16に設定します
)

# モデルを事前学習済みの状態から読み込みます。量子化設定とデバイスマップを指定します。
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")  # 自動デバイスマッピングでモデルを読み込みます
```

</div>
</details>

In [ ]:
import torch  # PyTorchライブラリをインポートします
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig  # Hugging FaceのTransformersライブラリから必要なクラスをインポートします

base_model_id = "mistralai/Mistral-7B-v0.1"  # 使用する基本モデルのIDを指定します
bnb_config = BitsAndBytesConfig(  # 量子化に関する設定を作成します
    load_in_4bit=True,  # 4ビットでモデルを読み込む設定をします
    bnb_4bit_use_double_quant=True,  # ダブル量子化を使用する設定を行います
    bnb_4bit_quant_type="nf4",  # 量子化のタイプを指定します（nf4を使用）
    bnb_4bit_compute_dtype=torch.bfloat16  # 計算データ型をbfloat16に設定します
)

# モデルを事前学習済みの状態から読み込みます。量子化設定とデバイスマップを指定します。
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")  # 自動デバイスマッピングでモデルを読み込みます

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token
```

</div>
<div class="column-right">

# 日本語訳

```python
tokenizer = AutoTokenizer.from_pretrained(  # 基本モデルのトークナイザーを読み込みます
    base_model_id,  # 使用するモデルIDを指定します
    model_max_length=512,  # モデルの最大長を512トークンに設定します
    padding_side="left",  # パディングを左側に設定します
    add_eos_token=True  # 終了トークンを追加する設定をします
)

# トークナイザーのパディングトークンを終了トークンに設定します
tokenizer.pad_token = tokenizer.eos_token  # パディングトークンを終了トークンと同じに設定することで、モデル入力の整合性を保ちます
```

</div>
</details>

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(  # 基本モデルのトークナイザーを読み込みます
    base_model_id,  # 使用するモデルIDを指定します
    model_max_length=512,  # モデルの最大長を512トークンに設定します
    padding_side="left",  # パディングを左側に設定します
    add_eos_token=True  # 終了トークンを追加する設定をします
)

# トークナイザーのパディングトークンを終了トークンに設定します
tokenizer.pad_token = tokenizer.eos_token  # パディングトークンを終了トークンと同じに設定することで、モデル入力の整合性を保ちます

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train_data = pd.read_csv('../input/lmsys-chatbot-arena/train.csv')
test_data = pd.read_csv('../input/lmsys-chatbot-arena/test.csv')
submission_data = pd.read_csv('../input/lmsys-chatbot-arena/sample_submission.csv')
```

</div>
<div class="column-right">

# 日本語訳

```python
# Pandasライブラリを使用して、CSVファイルを読み込みます
train_data = pd.read_csv('../input/lmsys-chatbot-arena/train.csv')  # トレーニングデータを読み込みます
test_data = pd.read_csv('../input/lmsys-chatbot-arena/test.csv')  # テストデータを読み込みます
submission_data = pd.read_csv('../input/lmsys-chatbot-arena/sample_submission.csv')  # 提出用サンプルデータを読み込みます
```

</div>
</details>

In [ ]:
# Pandasライブラリを使用して、CSVファイルを読み込みます
train_data = pd.read_csv('../input/lmsys-chatbot-arena/train.csv')  # トレーニングデータを読み込みます
test_data = pd.read_csv('../input/lmsys-chatbot-arena/test.csv')  # テストデータを読み込みます
submission_data = pd.read_csv('../input/lmsys-chatbot-arena/sample_submission.csv')  # 提出用サンプルデータを読み込みます

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train_data.shape,test_data.shape,submission_data.shape
```

</div>
<div class="column-right">

# 日本語訳

```python
# 各データセットの形状（行数と列数）を表示します
train_data.shape, test_data.shape, submission_data.shape  # トレーニングデータ、テストデータ、および提出データのサイズを確認します
```

</div>
</details>

In [ ]:
# 各データセットの形状（行数と列数）を表示します
train_data.shape, test_data.shape, submission_data.shape  # トレーニングデータ、テストデータ、および提出データのサイズを確認します

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

def trim_endings(custom_string):
    return custom_string[:-2][2:]

def count_newlines(custom_string):
    return custom_string.count('\\n')

def word_counts(custom_string):
    return len(custom_string.split())


def apply_transformations(df):
    
    df['prompt'] =  df['prompt'].map(process)
    df['response_a'] =  df['response_a'].map(process)
    df['response_b'] =  df['response_b'].map(process)
    
    df['prompt'] =  df['prompt'].map(trim_endings)
    df['response_a'] =  df['response_a'].map(trim_endings)
    df['response_b'] =  df['response_b'].map(trim_endings)
    
    df['res_a_line_count'] = df['response_a'].map(count_newlines).astype(str)
    df['res_b_line_count'] = df['response_b'].map(count_newlines).astype(str)
    
    df['res_a_word_count'] = df['response_a'].map(word_counts).astype(str)
    df['res_b_word_count'] = df['response_b'].map(word_counts).astype(str)
    
    return df

def create_summary(df):
    df['summary'] = 'User prompt: ' + df['prompt'] +  '\n\n Model A :\n' + df['response_a'] +'\n\n Model A length:\n' + df['res_a_word_count'] +'\n\n Model A Line Count:\n' + df['res_a_line_count'] +'\n\nModel B:\n'  + df['response_b'] +'\n\n \n\nModel B length:\n'  + df['res_b_word_count'] +'\n\n \n\nModel B Line Count:\n'  + df['res_b_line_count']
    return df

</div>
<div class="column-right">

# 日本語訳

```python
def process(input_str):  # 入力文字列を処理する関数
    stripped_str = input_str.strip('[]')  # 角括弧を削除します
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 各文を分離し、両端の引用符を削除します
    return  ' '.join(sentences)  # 文をスペースで結合して返します

def trim_endings(custom_string):  # 文字列の末尾をトリミングする関数
    return custom_string[:-2][2:]  # 最後の2文字を削除し、最初の2文字を除去して返します

def count_newlines(custom_string):  # 改行文字のカウントを行う関数
    return custom_string.count('\\n')  # 文字列内の改行数を返します

def word_counts(custom_string):  # 単語のカウントを行う関数
    return len(custom_string.split())  # 文字列を単語に分割し、その数を返します


def apply_transformations(df):  # データフレームに変換を適用する関数
    
    df['prompt'] =  df['prompt'].map(process)  # 'prompt'列にprocess関数を適用します
    df['response_a'] =  df['response_a'].map(process)  # 'response_a'列にprocess関数を適用します
    df['response_b'] =  df['response_b'].map(process)  # 'response_b'列にprocess関数を適用します
    
    df['prompt'] =  df['prompt'].map(trim_endings)  # 'prompt'列の末尾をトリミングします
    df['response_a'] =  df['response_a'].map(trim_endings)  # 'response_a'列の末尾をトリミングします
    df['response_b'] =  df['response_b'].map(trim_endings)  # 'response_b'列の末尾をトリミングします
    
    df['res_a_line_count'] = df['response_a'].map(count_newlines).astype(str)  # 'response_a'の改行数を計算し、新しい列に格納します
    df['res_b_line_count'] = df['response_b'].map(count_newlines).astype(str)  # 'response_b'の改行数を計算し、新しい列に格納します
    
    df['res_a_word_count'] = df['response_a'].map(word_counts).astype(str)  # 'response_a'の単語数を計算し、新しい列に格納します
    df['res_b_word_count'] = df['response_b'].map(word_counts).astype(str)  # 'response_b'の単語数を計算し、新しい列に格納します
    
    return df  # 変換後のデータフレームを返します

def create_summary(df):  # サマリーを作成する関数
    df['summary'] = 'User prompt: ' + df['prompt'] +  '\n\n Model A :\n' + df['response_a'] +'\n\n Model A length:\n' + df['res_a_word_count'] +'\n\n Model A Line Count:\n' + df['res_a_line_count'] +'\n\nModel B:\n'  + df['response_b'] +'\n\n \n\nModel B length:\n'  + df['res_b_word_count'] +'\n\n \n\nModel B Line Count:\n'  + df['res_b_line_count']
    return df  # データフレームにサマリー列を追加して返します
```

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train_data = apply_transformations(train_data)
train_data = create_summary(train_data)
test_data = apply_transformations(test_data)    
test_data = create_summary(test_data)
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニングデータに変換を適用し、サマリーを作成します
train_data = apply_transformations(train_data)  # トレーニングデータに変換を適用します
train_data = create_summary(train_data)  # トレーニングデータにサマリーを作成します

# テストデータにも同様に変換を適用し、サマリーを作成します
test_data = apply_transformations(test_data)  # テストデータに変換を適用します    
test_data = create_summary(test_data)  # テストデータにサマリーを作成します
```

</div>
</details>

In [ ]:
# トレーニングデータに変換を適用し、サマリーを作成します
train_data = apply_transformations(train_data)  # トレーニングデータに変換を適用します
train_data = create_summary(train_data)  # トレーニングデータにサマリーを作成します

# テストデータにも同様に変換を適用し、サマリーを作成します
test_data = apply_transformations(test_data)  # テストデータに変換を適用します    
test_data = create_summary(test_data)  # テストデータにサマリーを作成します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train_data['summary'][2]
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニングデータのサマリー列の3番目の要素を表示します
train_data['summary'][2]  # インデックス2のサマリーを取得して表示します
```

</div>
</details>

In [ ]:
# トレーニングデータのサマリー列の3番目の要素を表示します
train_data['summary'][2]  # インデックス2のサマリーを取得して表示します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train_data['labels'] = train_data[['winner_model_a','winner_model_b','winner_tie']].idxmax(axis=1)
train_data['labels']=train_data['label'].astype('category')
train_data['target']=train_data['label'].cat.codes
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニングデータにラベル列を追加します。最高スコアのモデルを特定します
train_data['labels'] = train_data[['winner_model_a', 'winner_model_b', 'winner_tie']].idxmax(axis=1)  # 各行で最大値を持つ列のインデックスを取得して'labels'列に格納します

train_data['labels'] = train_data['labels'].astype('category')  # 'labels'列をカテゴリ型に変換します
train_data['target'] = train_data['labels'].cat.codes  # 'labels'のカテゴリコードを' target'列に格納します
```

</div>
</details>

In [ ]:
# トレーニングデータにラベル列を追加します。最高スコアのモデルを特定します
train_data['labels'] = train_data[['winner_model_a', 'winner_model_b', 'winner_tie']].idxmax(axis=1)  # 各行で最大値を持つ列のインデックスを取得して'labels'列に格納します

train_data['labels'] = train_data['labels'].astype('category')  # 'labels'列をカテゴリ型に変換します
train_data['target'] = train_data['labels'].cat.codes  # 'labels'のカテゴリコードを' target'列に格納します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
category_map = {code: category for code, category in enumerate(train_data['labels'].cat.categories)}
category_map
```

</div>
<div class="column-right">

# 日本語訳

```python
# 'labels'列のカテゴリに対するコードを作成します
category_map = {code: category for code, category in enumerate(train_data['labels'].cat.categories)}  # 各カテゴリに対するコードを辞書形式で作成します
category_map  # カテゴリマップを表示します
```

</div>
</details>

In [ ]:
# 'labels'列のカテゴリに対するコードを作成します
category_map = {code: category for code, category in enumerate(train_data['labels'].cat.categories)}  # 各カテゴリに対するコードを辞書形式で作成します
category_map  # カテゴリマップを表示します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
SEED = 7920
```

</div>
<div class="column-right">

# 日本語訳

```python
# 再現性を持たせるためのランダムシードを設定します
SEED = 7920  # シード値を7920に設定します。これにより、結果が一貫性を持つようになります
```

</div>
</details>

In [ ]:
# 再現性を持たせるためのランダムシードを設定します
SEED = 7920  # シード値を7920に設定します。これにより、結果が一貫性を持つようになります

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score
train_df = train_data[['id','summary','target']]
test_df = train_data[['id','summary']]

df_train,df_val = train_test_split(train_df,test_size=0.2,random_state=SEED,stratify=train_df['target'])
```

</div>
<div class="column-right">

# 日本語訳

```python
from sklearn.datasets import make_classification  # データセットを生成するためのsklearnの関数をインポートします
from sklearn.model_selection import train_test_split  # データを訓練用と検証用に分割するための関数をインポートします
from sklearn.linear_model import LogisticRegression  # ロジスティック回帰モデルをインポートします
from sklearn.metrics import f1_score, confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score  # 評価指標を計算するための関数をインポートします

# 使用するカラムを選択してトレーニングデータとテストデータを準備します
train_df = train_data[['id', 'summary', 'target']]  # トレーニングデータから'id', 'summary', 'target'を選択します
test_df = train_data[['id', 'summary']]  # テストデータから'id'と'summary'を選択します

# トレーニングデータを訓練セットと検証セットに分割します
df_train, df_val = train_test_split(train_df, test_size=0.2, random_state=SEED, stratify=train_df['target'])  # 20%を検証データとして分割します。分類のバランスを保ちます
```

</div>
</details>

In [ ]:
from sklearn.datasets import make_classification  # データセットを生成するためのsklearnの関数をインポートします
from sklearn.model_selection import train_test_split  # データを訓練用と検証用に分割するための関数をインポートします
from sklearn.linear_model import LogisticRegression  # ロジスティック回帰モデルをインポートします
from sklearn.metrics import f1_score, confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score  # 評価指標を計算するための関数をインポートします

# 使用するカラムを選択してトレーニングデータとテストデータを準備します
train_df = train_data[['id', 'summary', 'target']]  # トレーニングデータから'id', 'summary', 'target'を選択します
test_df = train_data[['id', 'summary']]  # テストデータから'id'と'summary'を選択します

# トレーニングデータを訓練セットと検証セットに分割します
df_train, df_val = train_test_split(train_df, test_size=0.2, random_state=SEED, stratify=train_df['target'])  # 20%を検証データとして分割します。分類のバランスを保ちます

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from datasets import Dataset, DatasetDict
dataset_train = Dataset.from_pandas(df_train,preserve_index=False)
dataset_val = Dataset.from_pandas(df_val,preserve_index=False)
dataset_test = Dataset.from_pandas(test_df,preserve_index=False)
```

</div>
<div class="column-right">

# 日本語訳

```python
from datasets import Dataset, DatasetDict  # datasetsモジュールからDatasetおよびDatasetDictをインポートします

# PandasデータフレームからHugging FaceのDataset形式に変換します
dataset_train = Dataset.from_pandas(df_train, preserve_index=False)  # トレーニングデータをDatasetに変換します（インデックスは保存しません）
dataset_val = Dataset.from_pandas(df_val, preserve_index=False)  # 検証データをDatasetに変換します（インデックスは保存しません）
dataset_test = Dataset.from_pandas(test_df, preserve_index=False)  # テストデータをDatasetに変換します（インデックスは保存しません）
```

</div>
</details>

In [ ]:
from datasets import Dataset, DatasetDict  # datasetsモジュールからDatasetおよびDatasetDictをインポートします

# PandasデータフレームからHugging FaceのDataset形式に変換します
dataset_train = Dataset.from_pandas(df_train, preserve_index=False)  # トレーニングデータをDatasetに変換します（インデックスは保存しません）
dataset_val = Dataset.from_pandas(df_val, preserve_index=False)  # 検証データをDatasetに変換します（インデックスは保存しません）
dataset_test = Dataset.from_pandas(test_df, preserve_index=False)  # テストデータをDatasetに変換します（インデックスは保存しません）

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
 dataset_train_shuffled = dataset_train.shuffle(seed=SEED)
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニングデータセットをシャッフルして、新しいデータセットを作成します
dataset_train_shuffled = dataset_train.shuffle(seed=SEED)  # 指定したシードを使用してデータをランダムに並び替えます
```

</div>
</details>

In [ ]:
# トレーニングデータセットをシャッフルして、新しいデータセットを作成します
dataset_train_shuffled = dataset_train.shuffle(seed=SEED)  # 指定したシードを使用してデータをランダムに並び替えます

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
dataset = DatasetDict({
    'train': dataset_train_shuffled,
#     'val': dataset_val,
#     'test': dataset_test
})
dataset
```

</div>
<div class="column-right">

# 日本語訳

```python
# データセットを辞書形式でまとめて、トレーニング、検証、テストのデータセットを管理します
dataset = DatasetDict({  # データセットの辞書を作成します
    'train': dataset_train_shuffled,  # シャッフルしたトレーニングデータを追加します
    # 'val': dataset_val,  # 検証データを追加します（コメントアウト中）
    # 'test': dataset_test  # テストデータを追加します（コメントアウト中）
})
dataset  # 作成したデータセット辞書を表示します
```

</div>
</details>

In [ ]:
# データセットを辞書形式でまとめて、トレーニング、検証、テストのデータセットを管理します
dataset = DatasetDict({  # データセットの辞書を作成します
    'train': dataset_train_shuffled,  # シャッフルしたトレーニングデータを追加します
    # 'val': dataset_val,  # 検証データを追加します（コメントアウト中）
    # 'test': dataset_test  # テストデータを追加します（コメントアウト中）
})
dataset  # 作成したデータセット辞書を表示します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
df_train.target.value_counts(normalize=True)
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニングデータの'target'列の値のカウントを正規化して表示します
df_train.target.value_counts(normalize=True)  # 各クラスの割合を表示します。これにより、データのバランスを確認できます。
```

</div>
</details>

In [ ]:
# トレーニングデータの'target'列の値のカウントを正規化して表示します
df_train.target.value_counts(normalize=True)  # 各クラスの割合を表示します。これにより、データのバランスを確認できます。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
class_weights=(1/df_train.target.value_counts(normalize=True).sort_index()).tolist()
class_weights=torch.tensor(class_weights)
class_weights=class_weights/class_weights.sum()
class_weights
```

</div>
<div class="column-right">

# 日本語訳

```python
# クラスの重みを計算します。これにより、クラス間の不均衡を扱います
class_weights = (1 / df_train.target.value_counts(normalize=True).sort_index()).tolist()  # 正規化されたターゲットの値を使って、それらの逆数を計算します
class_weights = torch.tensor(class_weights)  # リストをPyTorchのテンソルに変換します
class_weights = class_weights / class_weights.sum()  # 重みを合計で割って、正規化された重みにします
class_weights  # 計算したクラス重みを表示します
```

</div>
</details>

In [ ]:
# クラスの重みを計算します。これにより、クラス間の不均衡を扱います
class_weights = (1 / df_train.target.value_counts(normalize=True).sort_index()).tolist()  # 正規化されたターゲットの値を使って、それらの逆数を計算します
class_weights = torch.tensor(class_weights)  # リストをPyTorchのテンソルに変換します
class_weights = class_weights / class_weights.sum()  # 重みを合計で割って、正規化された重みにします
class_weights  # 計算したクラス重みを表示します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token
```

</div>
<div class="column-right">

# 日本語訳

```python
# トークナイザーを再度設定します。これによりトークン化が可能になります
tokenizer = AutoTokenizer.from_pretrained(  # 基本モデルのトークナイザーを読み込みます
    base_model_id,  # 使用するモデルIDを指定します
    model_max_length=512,  # モデルの最大長を512トークンに設定します
    padding_side="left",  # パディングを左側に設定します
    add_eos_token=True  # 終了トークンを追加する設定をします
)

# トークナイザーのパディングトークンを終了トークンに設定します
tokenizer.pad_token = tokenizer.eos_token  # パディングトークンを終了トークンと同じに設定します
```

</div>
</details>

In [ ]:
# トークナイザーを再度設定します。これによりトークン化が可能になります
tokenizer = AutoTokenizer.from_pretrained(  # 基本モデルのトークナイザーを読み込みます
    base_model_id,  # 使用するモデルIDを指定します
    model_max_length=512,  # モデルの最大長を512トークンに設定します
    padding_side="left",  # パディングを左側に設定します
    add_eos_token=True  # 終了トークンを追加する設定をします
)

# トークナイザーのパディングトークンを終了トークンに設定します
tokenizer.pad_token = tokenizer.eos_token  # パディングトークンを終了トークンと同じに設定します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
MAX_LEN = 512
col_to_delete = ['summary']

def llama_preprocessing_function(examples):
    return tokenizer(examples['summary'], truncation=True, max_length=MAX_LEN)

tokenized_datasets = dataset.map(llama_preprocessing_function, batched=True, remove_columns=col_to_delete)
tokenized_datasets = tokenized_datasets.rename_column("target", "labels")
tokenized_datasets.set_format("torch")
```

</div>
<div class="column-right">

# 日本語訳

```python
MAX_LEN = 512  # モデルの最大入力長を512トークンに設定します
col_to_delete = ['summary']  # 削除する列のリストを定義します

def llama_preprocessing_function(examples):  # データを前処理する関数を定義します
    return tokenizer(examples['summary'], truncation=True, max_length=MAX_LEN)  # 'summary'カラムをトークン化し、長さを最大値でトランケーションします

# データセット全体に前処理関数を適用し、バッチ処理を行い、不要な列を削除します
tokenized_datasets = dataset.map(llama_preprocessing_function, batched=True, remove_columns=col_to_delete)

# 'target'列を'labels'列に名前を変更します
tokenized_datasets = tokenized_datasets.rename_column("target", "labels")

# データセットのフォーマットをPyTorch形式に設定します
tokenized_datasets.set_format("torch")  # トレーニング時にPyTorchが使用できる形式に設定します
```

</div>
</details>

In [ ]:
MAX_LEN = 512  # モデルの最大入力長を512トークンに設定します
col_to_delete = ['summary']  # 削除する列のリストを定義します

def llama_preprocessing_function(examples):  # データを前処理する関数を定義します
    return tokenizer(examples['summary'], truncation=True, max_length=MAX_LEN)  # 'summary'カラムをトークン化し、長さを最大値でトランケーションします

# データセット全体に前処理関数を適用し、バッチ処理を行い、不要な列を削除します
tokenized_datasets = dataset.map(llama_preprocessing_function, batched=True, remove_columns=col_to_delete)

# 'target'列を'labels'列に名前を変更します
tokenized_datasets = tokenized_datasets.rename_column("target", "labels")

# データセットのフォーマットをPyTorch形式に設定します
tokenized_datasets.set_format("torch")  # トレーニング時にPyTorchが使用できる形式に設定します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
```

</div>
<div class="column-right">

# 日本語訳

```python
from peft import prepare_model_for_kbit_training  # PEFTからkビットトレーニング用のモデル準備関数をインポートします

model.gradient_checkpointing_enable()  # 勾配チェックポイントを有効にして、メモリを節約します
model = prepare_model_for_kbit_training(model)  # モデルをkビットトレーニング用に準備します
```

</div>
</details>

In [ ]:
from peft import prepare_model_for_kbit_training  # PEFTからkビットトレーニング用のモデル準備関数をインポートします

model.gradient_checkpointing_enable()  # 勾配チェックポイントを有効にして、メモリを節約します
model = prepare_model_for_kbit_training(model)  # モデルをkビットトレーニング用に準備します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

```

</div>
<div class="column-right">

# 日本語訳

```python
def print_trainable_parameters(model):  # モデルのトレーニング可能なパラメータを表示する関数
    """
    モデル内のトレーニング可能なパラメータの数を表示します。
    """
    trainable_params = 0  # トレーニング可能なパラメータの初期化
    all_param = 0  # すべてのパラメータの初期化
    for _, param in model.named_parameters():  # モデル内のすべてのパラメータをループします
        all_param += param.numel()  # 総パラメータ数を計算します
        if param.requires_grad:  # パラメータが勾配計算を必要とする場合
            trainable_params += param.numel()  # トレーニング可能なパラメータ数を累積します
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )  # トレーニング可能なパラメータ数、すべてのパラメータ数、トレーニング可能なパラメータの割合を表示します
```

</div>
</details>

In [ ]:
def print_trainable_parameters(model):  # モデルのトレーニング可能なパラメータを表示する関数
    """
    モデル内のトレーニング可能なパラメータの数を表示します。
    """
    trainable_params = 0  # トレーニング可能なパラメータの初期化
    all_param = 0  # すべてのパラメータの初期化
    for _, param in model.named_parameters():  # モデル内のすべてのパラメータをループします
        all_param += param.numel()  # 総パラメータ数を計算します
        if param.requires_grad:  # パラメータが勾配計算を必要とする場合
            trainable_params += param.numel()  # トレーニング可能なパラメータ数を累積します
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )  # トレーニング可能なパラメータ数、すべてのパラメータ数、トレーニング可能なパラメータの割合を表示します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
print(model)
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルの詳細を表示します
print(model)  # モデルの構造とパラメータの詳細を出力します
```

</div>
</details>

In [ ]:
# モデルの詳細を表示します
print(model)  # モデルの構造とパラメータの詳細を出力します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)
```

</div>
<div class="column-right">

# 日本語訳

```python
from accelerate import FullyShardedDataParallelPlugin, Accelerator  # Accelerateライブラリから関連するクラスをインポートします
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig  # FSDPのための設定クラスをインポートします

# Fully Sharded Data Parallelプラグインを設定します
fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),  # チェックポイントをCPUにオフロードする設定
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),  # 最適化ステート情報もCPUにオフロードする設定
)

# Acceleratorインスタンスを作成します
accelerator = Accelerator(fsdp_plugin=fsdp_plugin)  # FSDPプラグインを使用してAcceleratorを初期化します
```

</div>
</details>

In [ ]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator  # Accelerateライブラリから関連するクラスをインポートします
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig  # FSDPのための設定クラスをインポートします

# Fully Sharded Data Parallelプラグインを設定します
fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),  # チェックポイントをCPUにオフロードする設定
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),  # 最適化ステート情報もCPUにオフロードする設定
)

# Acceleratorインスタンスを作成します
accelerator = Accelerator(fsdp_plugin=fsdp_plugin)  # FSDPプラグインを使用してAcceleratorを初期化します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

# Apply the accelerator. You can comment this out to remove the accelerator.
model = accelerator.prepare_model(model)
```

</div>
<div class="column-right">

# 日本語訳

```python
from peft import LoraConfig, get_peft_model  # PEFTライブラリからLoraConfigとモデル取得関数をインポートします

# LoRAの設定を定義します
config = LoraConfig(
    r=8,  # rankの値
    lora_alpha=16,  # LoRAのスケーリングファクター
    target_modules=[  # 対象モジュールのリスト
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",  # バイアスの設定
    lora_dropout=0.05,  # LoRAのドロップアウト率
    task_type="CAUSAL_LM",  # タスクタイプを指定します（因果言語モデル）
)

# モデルにLoRA設定を適用します
model = get_peft_model(model, config)  # LoRAの設定を持つモデルを取得します
print_trainable_parameters(model)  # トレーニング可能なパラメータを表示します

# アクセラレーターを適用します。必要に応じてコメントアウトできます。
model = accelerator.prepare_model(model)  # Accelerateを使用してモデルを準備します
```

</div>
</details>

In [ ]:
from peft import LoraConfig, get_peft_model  # PEFTライブラリからLoraConfigとモデル取得関数をインポートします

# LoRAの設定を定義します
config = LoraConfig(
    r=8,  # rankの値
    lora_alpha=16,  # LoRAのスケーリングファクター
    target_modules=[  # 対象モジュールのリスト
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",  # バイアスの設定
    lora_dropout=0.05,  # LoRAのドロップアウト率
    task_type="CAUSAL_LM",  # タスクタイプを指定します（因果言語モデル）
)

# モデルにLoRA設定を適用します
model = get_peft_model(model, config)  # LoRAの設定を持つモデルを取得します
print_trainable_parameters(model)  # トレーニング可能なパラメータを表示します

# アクセラレーターを適用します。必要に応じてコメントアウトできます。
model = accelerator.prepare_model(model)  # Accelerateを使用してモデルを準備します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install -q wandb -U

import wandb, os
wandb.login()

wandb_project = "mistral-finetune"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project
```

</div>
<div class="column-right">

# 日本語訳

```python
# Weights & Biases（WandB）ライブラリをインストールします
!pip install -q wandb -U  # WandBをアップグレードします

import wandb, os  # WandBとOSモジュールをインポートします
wandb.login()  # WandBにログインします

wandb_project = "mistral-finetune"  # プロジェクト名を設定します
if len(wandb_project) > 0:  # プロジェクト名が空でない場合
    os.environ["WANDB_PROJECT"] = wandb_project  # 環境変数にプロジェクト名を設定します
```

</div>
</details>

In [ ]:
# Weights & Biases（WandB）ライブラリをインストールします
!pip install -q wandb -U  # WandBをアップグレードします

import wandb, os  # WandBとOSモジュールをインポートします
wandb.login()  # WandBにログインします

wandb_project = "mistral-finetune"  # プロジェクト名を設定します
if len(wandb_project) > 0:  # プロジェクト名が空でない場合
    os.environ["WANDB_PROJECT"] = wandb_project  # 環境変数にプロジェクト名を設定します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True
```

</div>
<div class="column-right">

# 日本語訳

```python
# GPUの数が1つより多い場合の処理を行います
if torch.cuda.device_count() > 1:  # 使用可能なGPUが1つ以上の場合
    model.is_parallelizable = True  # モデルが並列化可能であることを指定します
    model.model_parallel = True  # モデル並列を有効にします
```

</div>
</details>

In [ ]:
# GPUの数が1つより多い場合の処理を行います
if torch.cuda.device_count() > 1:  # 使用可能なGPUが1つ以上の場合
    model.is_parallelizable = True  # モデルが並列化可能であることを指定します
    model.model_parallel = True  # モデル並列を有効にします

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!nvcc --version
!pip uninstall torch torchvision torchaudio -y
!pip install torch==2.0.0+cu121 torchvision==0.15.0+cu121 torchaudio==2.0.0+cu121 -f https://download.pytorch.org/whl/nightly/cu121/torch_nightly.html

```

</div>
<div class="column-right">

# 日本語訳

```python
# NVIDIAのCUDAコンパイラのバージョンを表示します
!nvcc --version  # nvccのバージョン情報を表示します

# 古いバージョンのPyTorch関連ライブラリをアンインストールします
!pip uninstall torch torchvision torchaudio -y  # PyTorch、TorchVision、およびTorchAudioを強制的にアンインストールします

# CUDA 12.1対応の指定バージョンのPyTorch関連ライブラリをインストールします
!pip install torch==2.0.0+cu121 torchvision==0.15.0+cu121 torchaudio==2.0.0+cu121 -f https://download.pytorch.org/whl/nightly/cu121/torch_nightly.html  # PyTorchの特定バージョンをインストールします
```

</div>
</details>

In [ ]:
# NVIDIAのCUDAコンパイラのバージョンを表示します
!nvcc --version  # nvccのバージョン情報を表示します

# 古いバージョンのPyTorch関連ライブラリをアンインストールします
!pip uninstall torch torchvision torchaudio -y  # PyTorch、TorchVision、およびTorchAudioを強制的にアンインストールします

# CUDA 12.1対応の指定バージョンのPyTorch関連ライブラリをインストールします
!pip install torch==2.0.0+cu121 torchvision==0.15.0+cu121 torchaudio==2.0.0+cu121 -f https://download.pytorch.org/whl/nightly/cu121/torch_nightly.html  # PyTorchの特定バージョンをインストールします

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Uninstall existing versions of torch, torchvision, and torchaudio
!pip uninstall torch torchvision torchaudio -y

# Install compatible versions with CUDA 11.2 support
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu112/torch_stable.html

```

</div>
<div class="column-right">

# 日本語訳

```python
# 既存のtorch、torchvision、torchaudioのバージョンをアンインストールします
!pip uninstall torch torchvision torchaudio -y  # PyTorch、TorchVision、およびTorchAudioを強制的にアンインストールします

# CUDA 11.2に対応した互換性のあるバージョンをインストールします
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu112/torch_stable.html  # PyTorchの安定したバージョンをインストールします
```

</div>
</details>

In [ ]:
# 既存のtorch、torchvision、torchaudioのバージョンをアンインストールします
!pip uninstall torch torchvision torchaudio -y  # PyTorch、TorchVision、およびTorchAudioを強制的にアンインストールします

# CUDA 11.2に対応した互換性のあるバージョンをインストールします
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu112/torch_stable.html  # PyTorchの安定したバージョンをインストールします

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'

```

</div>
<div class="column-right">

# 日本語訳

```python
import os  # OSモジュールをインポートします

# CUDAの起動ブロッキングを無効にします
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'  # CUDAカーネルの起動時にブロッキングを無効にします。これにより、実行時のパフォーマンスが向上する可能性があります。
```

</div>
</details>

In [ ]:
import os  # OSモジュールをインポートします

# CUDAの起動ブロッキングを無効にします
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'  # CUDAカーネルの起動時にブロッキングを無効にします。これにより、実行時のパフォーマンスが向上する可能性があります。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import torch

seed = 7920  # Replace with your desired seed

if torch.cuda.is_available():
    generator = torch.Generator(device='cuda').manual_seed(seed)
else:
    generator = torch.Generator().manual_seed(seed)

```

</div>
<div class="column-right">

# 日本語訳

```python
import torch  # PyTorchライブラリをインポートします

seed = 7920  # 使用するシード値を設定します

# CUDAが使用可能であれば、CUDAデバイス用の乱数生成器を設定します
if torch.cuda.is_available():
    generator = torch.Generator(device='cuda').manual_seed(seed)  # CUDAデバイス用にシードを設定します
else:
    generator = torch.Generator().manual_seed(seed)  # CPU用にシードを設定します
```

</div>
</details>

In [ ]:
import torch  # PyTorchライブラリをインポートします

seed = 7920  # 使用するシード値を設定します

# CUDAが使用可能であれば、CUDAデバイス用の乱数生成器を設定します
if torch.cuda.is_available():
    generator = torch.Generator(device='cuda').manual_seed(seed)  # CUDAデバイス用にシードを設定します
else:
    generator = torch.Generator().manual_seed(seed)  # CPU用にシードを設定します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
if torch.cuda.is_available():
  generator = torch.Generator('cuda').manual_seed(seed)
else:
  generator = torch.Generator().manual_seed(seed)
```

</div>
<div class="column-right">

# 日本語訳

```python
# CUDAが利用可能であれば、CUDAデバイス用の乱数生成器を設定します
if torch.cuda.is_available():  
    generator = torch.Generator('cuda').manual_seed(seed)  # CUDAデバイスの乱数生成器をシードで初期化します
else:
    generator = torch.Generator().manual_seed(seed)  # CPU用の乱数生成器をシードで初期化します
```

</div>
</details>

In [ ]:
# CUDAが利用可能であれば、CUDAデバイス用の乱数生成器を設定します
if torch.cuda.is_available():  
    generator = torch.Generator('cuda').manual_seed(seed)  # CUDAデバイスの乱数生成器をシードで初期化します
else:
    generator = torch.Generator().manual_seed(seed)  # CPU用の乱数生成器をシードで初期化します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import transformers
class CustomTrainer(transformers.Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        # Ensure label_weights is a tensor
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False):
        # Extract labels and convert them to long type for cross_entropy
        labels = inputs.pop("labels").long()

        # Forward pass
        outputs = model(**inputs)

        # Extract logits assuming they are directly outputted by the model
        logits = outputs.get('logits')

        # Compute custom loss with class weights for imbalanced data handling
        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)
        else:
            loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss
```

</div>
<div class="column-right">

# 日本語訳

```python
import transformers  # Transformersライブラリをインポートします

class CustomTrainer(transformers.Trainer):  # Trainerクラスを継承したカスタムトレーナークラスを定義します
    def __init__(self, *args, class_weights=None, **kwargs):  # コンストラクタを定義します
        super().__init__(*args, **kwargs)  # 親クラスのコンストラクタを呼び出します
        # クラス重みが与えられた場合、テンソルに変換します
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)  # クラス重みをデバイスに移動します
        else:
            self.class_weights = None  # クラス重みがない場合はNoneとします

    def compute_loss(self, model, inputs, return_outputs=False):  # 損失を計算する関数を定義します
        # ラベルを抽出し、cross_entropy用にlong型に変換します
        labels = inputs.pop("labels").long()  # 入力からラベルを取り出します

        # フォワードパス
        outputs = model(**inputs)  # モデルを介して入力を処理します

        # 出力されたロジットを抽出します
        logits = outputs.get('logits')  # モデル出力からロジットを取り出します

        # 不均衡データ処理のためのクラス重みを用いたカスタム損失を計算します
        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)  # クラス重みを使った損失計算
        else:
            loss = F.cross_entropy(logits, labels)  # クラス重みなしで損失計算

        return (loss, outputs) if return_outputs else loss  # 出力の条件に応じて損失と出力を返します
```

</div>
</details>

In [ ]:
import transformers  # Transformersライブラリをインポートします

class CustomTrainer(transformers.Trainer):  # Trainerクラスを継承したカスタムトレーナークラスを定義します
    def __init__(self, *args, class_weights=None, **kwargs):  # コンストラクタを定義します
        super().__init__(*args, **kwargs)  # 親クラスのコンストラクタを呼び出します
        # クラス重みが与えられた場合、テンソルに変換します
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)  # クラス重みをデバイスに移動します
        else:
            self.class_weights = None  # クラス重みがない場合はNoneとします

    def compute_loss(self, model, inputs, return_outputs=False):  # 損失を計算する関数を定義します
        # ラベルを抽出し、cross_entropy用にlong型に変換します
        labels = inputs.pop("labels").long()  # 入力からラベルを取り出します

        # フォワードパス
        outputs = model(**inputs)  # モデルを介して入力を処理します

        # 出力されたロジットを抽出します
        logits = outputs.get('logits')  # モデル出力からロジットを取り出します

        # 不均衡データ処理のためのクラス重みを用いたカスタム損失を計算します
        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)  # クラス重みを使った損失計算
        else:
            loss = F.cross_entropy(logits, labels)  # クラス重みなしで損失計算

        return (loss, outputs) if return_outputs else loss  # 出力の条件に応じて損失と出力を返します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'balanced_accuracy' : balanced_accuracy_score(predictions, labels),'accuracy':accuracy_score(predictions,labels)}
```

</div>
<div class="column-right">

# 日本語訳

```python
def compute_metrics(eval_pred):  # 評価予測をもとにメトリクスを計算する関数を定義します
    predictions, labels = eval_pred  # 評価予測から予測値とラベルを取得します
    predictions = np.argmax(predictions, axis=1)  # 予測の中で最大値のインデックスを取得してクラスラベルを決定します
    # 精度とバランス精度を計算して返します
    return {
        'balanced_accuracy': balanced_accuracy_score(predictions, labels),  # バランス精度を計算します
        'accuracy': accuracy_score(predictions, labels)  # 純粋な精度を計算します
    }
```

</div>
</details>

In [ ]:
def compute_metrics(eval_pred):  # 評価予測をもとにメトリクスを計算する関数を定義します
    predictions, labels = eval_pred  # 評価予測から予測値とラベルを取得します
    predictions = np.argmax(predictions, axis=1)  # 予測の中で最大値のインデックスを取得してクラスラベルを決定します
    # 精度とバランス精度を計算して返します
    return {
        'balanced_accuracy': balanced_accuracy_score(predictions, labels),  # バランス精度を計算します
        'accuracy': accuracy_score(predictions, labels)  # 純粋な精度を計算します
    }

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
training_args = transformers.TrainingArguments(
    output_dir = 'model_classification',
    learning_rate = 1e-4,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 1,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True,
	logging_steps=10,
	max_steps=100,
    fp16=True,
    push_to_hub=False
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニングに関する引数を設定します
training_args = transformers.TrainingArguments(
    output_dir='model_classification',  # モデルの出力先ディレクトリ
    learning_rate=1e-4,  # 学習率
    per_device_train_batch_size=8,  # デバイスごとのトレーニングバッチサイズ
    per_device_eval_batch_size=8,  # デバイスごとの評価バッチサイズ
    num_train_epochs=1,  # トレーニングエポック数
    weight_decay=0.01,  # 重み減衰率
    evaluation_strategy='epoch',  # 評価戦略をエポック単位に設定
    save_strategy='epoch',  # モデルの保存戦略をエポック単位に設定
    load_best_model_at_end=True,  # 最良のモデルを最後にロード
	logging_steps=10,  # ロギングの頻度
	max_steps=100,  # 最大ステップ数
    fp16=True,  # 16ビット浮動小数点でトレーニングを実行
    push_to_hub=False  # モデルをHugging Face Hubにプッシュしない設定
)
```

</div>
</details>

In [ ]:
# トレーニングに関する引数を設定します
training_args = transformers.TrainingArguments(
    output_dir='model_classification',  # モデルの出力先ディレクトリ
    learning_rate=1e-4,  # 学習率
    per_device_train_batch_size=8,  # デバイスごとのトレーニングバッチサイズ
    per_device_eval_batch_size=8,  # デバイスごとの評価バッチサイズ
    num_train_epochs=1,  # トレーニングエポック数
    weight_decay=0.01,  # 重み減衰率
    evaluation_strategy='epoch',  # 評価戦略をエポック単位に設定
    save_strategy='epoch',  # モデルの保存戦略をエポック単位に設定
    load_best_model_at_end=True,  # 最良のモデルを最後にロード
	logging_steps=10,  # ロギングの頻度
	max_steps=100,  # 最大ステップ数
    fp16=True,  # 16ビット浮動小数点でトレーニングを実行
    push_to_hub=False  # モデルをHugging Face Hubにプッシュしない設定
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
collate_fn = transformers.DataCollatorWithPadding(tokenizer=tokenizer)
```

</div>
<div class="column-right">

# 日本語訳

```python
# データのコラテーションを行うための関数を設定します
collate_fn = transformers.DataCollatorWithPadding(tokenizer=tokenizer)  # トークナイザーを用いてパディング付きのデータコレータを作成します
```

</div>
</details>

In [ ]:
# データのコラテーションを行うための関数を設定します
collate_fn = transformers.DataCollatorWithPadding(tokenizer=tokenizer)  # トークナイザーを用いてパディング付きのデータコレータを作成します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    tokenizer = tokenizer,
    data_collator = collate_fn,
    compute_metrics = compute_metrics,
    class_weights=class_weights,
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# カスタムトレーナーのインスタンスを作成します
trainer = CustomTrainer(
    model=model,  # 使用するモデル
    args=training_args,  # トレーニング引数
    train_dataset=tokenized_datasets['train'],  # トレーニングデータセット
    tokenizer=tokenizer,  # トークナイザー
    data_collator=collate_fn,  # データコレータ
    compute_metrics=compute_metrics,  # メトリクス計算関数
    class_weights=class_weights,  # クラス重み
)
```

</div>
</details>

In [ ]:
# カスタムトレーナーのインスタンスを作成します
trainer = CustomTrainer(
    model=model,  # 使用するモデル
    args=training_args,  # トレーニング引数
    train_dataset=tokenized_datasets['train'],  # トレーニングデータセット
    tokenizer=tokenizer,  # トークナイザー
    data_collator=collate_fn,  # データコレータ
    compute_metrics=compute_metrics,  # メトリクス計算関数
    class_weights=class_weights,  # クラス重み
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install torch
```

</div>
<div class="column-right">

# 日本語訳

```python
# PyTorchライブラリをインストールします
!pip install torch  # 最新のPyTorchをインストールします
```

</div>
</details>

In [ ]:
# PyTorchライブラリをインストールします
!pip install torch  # 最新のPyTorchをインストールします

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import transformers
from datetime import datetime
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Check if CUDA is available
print("CUDA Available:", torch.cuda.is_available())
project = "mistral-finetune"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_datasets['train'],
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=2,
        gradient_checkpointing=True,
        gradient_accumulation_steps=4,
        max_steps=1000,
        learning_rate=2.5e-5, # Want about 10x smaller than the Mistral learning rate
        logging_steps=50,
        fp16=False,
        optim="paged_adamw_8bit",
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=50,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=50,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
        report_to="wandb",           # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()
```

</div>
<div class="column-right">

# 日本語訳

```python
import transformers  # Transformersライブラリをインポートします
from datetime import datetime  # 日付時刻を扱うためのライブラリをインポートします
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # CUDAカーネルの起動をブロッキングします

# CUDAの利用可能性を確認します
print("CUDA Available:", torch.cuda.is_available())  # CUDAが使用可能かどうかを表示します
project = "mistral-finetune"  # プロジェクト名を設定します
base_model_name = "mistral"  # 基本モデル名を設定します
run_name = base_model_name + "-" + project  # 実行名を設定します
output_dir = "./" + run_name  # 出力ディレクトリを設定します

tokenizer.pad_token = tokenizer.eos_token  # パディングトークンを終了トークンに設定します

# Trainerインスタンスを作成します
trainer = transformers.Trainer(
    model=model,  # 使用するモデル
    train_dataset=tokenized_datasets['train'],  # トレーニングデータセット
    args=transformers.TrainingArguments(
        output_dir=output_dir,  # 出力ディレクトリ
        warmup_steps=5,  # ウォームアップステップ数
        per_device_train_batch_size=2,  # デバイスごとのトレーニングバッチサイズ
        gradient_checkpointing=True,  # 勾配チェックポイントを有効にします
        gradient_accumulation_steps=4,  # 勾配蓄積ステップ数
        max_steps=1000,  # 最大ステップ数
        learning_rate=2.5e-5,  # 学習率
        logging_steps=50,  # ロギングの頻度
        fp16=False,  # 16ビット浮動小数点を無効にします
        optim="paged_adamw_8bit",  # 最適化手法を指定します
        logging_dir="./logs",  # ログの保存先ディレクトリ
        save_strategy="steps",  # ステップごとにモデルチェックポイントを保存
        save_steps=50,  # 50ステップごとにチェックポイントを保存
        evaluation_strategy="steps",  # ステップごとに評価
        eval_steps=50,  # 50ステップごとに評価しチェックポイントを保存
        do_eval=True,  # トレーニングの終了時に評価を実行
        report_to="wandb",  # Weights & Biasesにレポートするかどうか
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}",  # W&Bランの名前の設定
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),  # 言語モデル用のデータコラレータを設定
)

model.config.use_cache = False  # 警告を抑制します。推論時には再度有効にしてください
trainer.train()  # トレーニングを開始します
```

</div>
</details>

In [ ]:
import transformers  # Transformersライブラリをインポートします
from datetime import datetime  # 日付時刻を扱うためのライブラリをインポートします
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # CUDAカーネルの起動をブロッキングします

# CUDAの利用可能性を確認します
print("CUDA Available:", torch.cuda.is_available())  # CUDAが使用可能かどうかを表示します
project = "mistral-finetune"  # プロジェクト名を設定します
base_model_name = "mistral"  # 基本モデル名を設定します
run_name = base_model_name + "-" + project  # 実行名を設定します
output_dir = "./" + run_name  # 出力ディレクトリを設定します

tokenizer.pad_token = tokenizer.eos_token  # パディングトークンを終了トークンに設定します

# Trainerインスタンスを作成します
trainer = transformers.Trainer(
    model=model,  # 使用するモデル
    train_dataset=tokenized_datasets['train'],  # トレーニングデータセット
    args=transformers.TrainingArguments(
        output_dir=output_dir,  # 出力ディレクトリ
        warmup_steps=5,  # ウォームアップステップ数
        per_device_train_batch_size=2,  # デバイスごとのトレーニングバッチサイズ
        gradient_checkpointing=True,  # 勾配チェックポイントを有効にします
        gradient_accumulation_steps=4,  # 勾配蓄積ステップ数
        max_steps=1000,  # 最大ステップ数
        learning_rate=2.5e-5,  # 学習率
        logging_steps=50,  # ロギングの頻度
        fp16=False,  # 16ビット浮動小数点を無効にします
        optim="paged_adamw_8bit",  # 最適化手法を指定します
        logging_dir="./logs",  # ログの保存先ディレクトリ
        save_strategy="steps",  # ステップごとにモデルチェックポイントを保存
        save_steps=50,  # 50ステップごとにチェックポイントを保存
        evaluation_strategy="steps",  # ステップごとに評価
        eval_steps=50,  # 50ステップごとに評価しチェックポイントを保存
        do_eval=True,  # トレーニングの終了時に評価を実行
        report_to="wandb",  # Weights & Biasesにレポートするかどうか
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}",  # W&Bランの名前の設定
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),  # 言語モデル用のデータコラレータを設定
)

model.config.use_cache = False  # 警告を抑制します。推論時には再度有効にしてください
trainer.train()  # トレーニングを開始します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
    trust_remote_code=True,
)
```

</div>
<div class="column-right">

# 日本語訳

```python
import torch  # PyTorchライブラリをインポートします
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig  # 必要なクラスをインポートします

base_model_id = "mistralai/Mistral-7B-v0.1"  # 使用する基本モデルのIDを指定します
bnb_config = BitsAndBytesConfig(  # 量子化に関する設定を作成します
    load_in_4bit=True,  # 4ビットでモデルを読み込む設定
    bnb_4bit_use_double_quant=True,  # 2重量子化を使用する設定
    bnb_4bit_quant_type="nf4",  # 量子化のタイプを指定します（nf4）
    bnb_4bit_compute_dtype=torch.bfloat16  # 計算データ型をbfloat16に設定
)

# 事前学習済みのモデルを読み込みます
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistralモデル
    quantization_config=bnb_config,  # 量子化設定
    device_map="auto",  # 自動デバイスマッピング
    trust_remote_code=True,  # リモートコードを信頼する設定
)

# 評価用のトークナイザーを読み込みます
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,  # 使用するモデルIDを指定します
    add_bos_token=True,  # 始まりのトークンを追加する設定
    trust_remote_code=True,  # リモートコードを信頼する設定
)
```

</div>
</details>

In [ ]:
import torch  # PyTorchライブラリをインポートします
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig  # 必要なクラスをインポートします

base_model_id = "mistralai/Mistral-7B-v0.1"  # 使用する基本モデルのIDを指定します
bnb_config = BitsAndBytesConfig(  # 量子化に関する設定を作成します
    load_in_4bit=True,  # 4ビットでモデルを読み込む設定
    bnb_4bit_use_double_quant=True,  # 2重量子化を使用する設定
    bnb_4bit_quant_type="nf4",  # 量子化のタイプを指定します（nf4）
    bnb_4bit_compute_dtype=torch.bfloat16  # 計算データ型をbfloat16に設定
)

# 事前学習済みのモデルを読み込みます
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistralモデル
    quantization_config=bnb_config,  # 量子化設定
    device_map="auto",  # 自動デバイスマッピング
    trust_remote_code=True,  # リモートコードを信頼する設定
)

# 評価用のトークナイザーを読み込みます
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,  # 使用するモデルIDを指定します
    add_bos_token=True,  # 始まりのトークンを追加する設定
    trust_remote_code=True,  # リモートコードを信頼する設定
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "mistral-viggo-finetune/checkpoint-1000")
```

</div>
<div class="column-right">

# 日本語訳

```python
from peft import PeftModel  # PEFTライブラリからPeftModelをインポートします

# 事前学習済みのモデルにファインチューニングされたモデルをロードします
ft_model = PeftModel.from_pretrained(base_model, "mistral-viggo-finetune/checkpoint-1000")  # 指定されたチェックポイントからファインチューニングモデルを読み込みます
```

</div>
</details>

In [ ]:
from peft import PeftModel  # PEFTライブラリからPeftModelをインポートします

# 事前学習済みのモデルにファインチューニングされたモデルをロードします
ft_model = PeftModel.from_pretrained(base_model, "mistral-viggo-finetune/checkpoint-1000")  # 指定されたチェックポイントからファインチューニングモデルを読み込みます

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
eval_prompt = """Given a target sentence construct the underlying meaning representation of the input sentence as a single function with attributes and attribute values.
This function should describe the target string accurately and the function must be one of the following ['inform', 'request', 'give_opinion', 'confirm', 'verify_attribute', 'suggest', 'request_explanation', 'recommend', 'request_attribute'].
The attributes must be one of the following: ['name', 'exp_release_date', 'release_year', 'developer', 'esrb', 'rating', 'genres', 'player_perspective', 'has_multiplayer', 'platforms', 'available_on_steam', 'has_linux_release', 'has_mac_release', 'specifier']

### Target sentence:
Earlier, you stated that you didn't have strong feelings about PlayStation's Little Big Adventure. Is your opinion true for all games which don't have multiplayer?

### Meaning representation:
"""

model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
```

</div>
<div class="column-right">

# 日本語訳

```python
# 評価用のプロンプトを定義します
eval_prompt = """Given a target sentence construct the underlying meaning representation of the input sentence as a single function with attributes and attribute values.
This function should describe the target string accurately and the function must be one of the following ['inform', 'request', 'give_opinion', 'confirm', 'verify_attribute', 'suggest', 'request_explanation', 'recommend', 'request_attribute'].
The attributes must be one of the following: ['name', 'exp_release_date', 'release_year', 'developer', 'esrb', 'rating', 'genres', 'player_perspective', 'has_multiplayer', 'platforms', 'available_on_steam', 'has_linux_release', 'has_mac_release', 'specifier']

### Target sentence:
Earlier, you stated that you didn't have strong feelings about PlayStation's Little Big Adventure. Is your opinion true for all games which don't have multiplayer?

### Meaning representation:
""" 

# トークナイザーを使用してモデル入力を準備します
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")  # CUDAデバイスに転送します

# モデルを評価モードに設定します
ft_model.eval()
with torch.no_grad():  # 勾配計算を無効にします
    # モデルを使って生成した出力を表示します
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))  # 生成されたトークンをデコードし、特殊トークンをスキップして表示します
```

</div>
</details>

In [ ]:
# 評価用のプロンプトを定義します
eval_prompt = """Given a target sentence construct the underlying meaning representation of the input sentence as a single function with attributes and attribute values.
This function should describe the target string accurately and the function must be one of the following ['inform', 'request', 'give_opinion', 'confirm', 'verify_attribute', 'suggest', 'request_explanation', 'recommend', 'request_attribute'].
The attributes must be one of the following: ['name', 'exp_release_date', 'release_year', 'developer', 'esrb', 'rating', 'genres', 'player_perspective', 'has_multiplayer', 'platforms', 'available_on_steam', 'has_linux_release', 'has_mac_release', 'specifier']

### Target sentence:
Earlier, you stated that you didn't have strong feelings about PlayStation's Little Big Adventure. Is your opinion true for all games which don't have multiplayer?

### Meaning representation:
""" 

# トークナイザーを使用してモデル入力を準備します
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")  # CUDAデバイスに転送します

# モデルを評価モードに設定します
ft_model.eval()
with torch.no_grad():  # 勾配計算を無効にします
    # モデルを使って生成した出力を表示します
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))  # 生成されたトークンをデコードし、特殊トークンをスキップして表示します